In [3]:
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
from scipy.stats import norm, t
from FES import FES
import random
from collections import deque
from collections import defaultdict

In [4]:
class Vehicles:
    def __init__(self,origin, destination, vehicle_type, path,u,w,idx, reroutes = False):
        self.origin = origin
        self.destination  = destination
        self.vehicle_type = vehicle_type
        self.v_max = 100 if vehicle_type == 'car' else 80
        self.path = path
        self.curren_position = origin
        self.reroutes = reroutes

        self.travel_time = 0
        self.delayed_time = 0
        self.incident_count = 0
        
        
        self.link_idx = idx
        
    def update_position(self, current_time,fes, veh_on_link,G):
        if self.path[self.link_idx] == self.destination:
            fes.add(Event(current_time, 'departure',self))
            return
        
        u = self.path[self.link_idx]
        w = self.path[self.link_idx + 1]
        self.current_link = (u,w)
        self.link_entry_time = current_time

        distance = G[u][w]['length']
        travel_time = calculate_travel_time(distance, self.v_max)
        self.estimated_exit_time = current_time + travel_time


        veh_on_link[(u, w)].append(self)

        # Schedule the next move
        fes.add(Event(self.estimated_exit_time, 'move_next_link', self))

    def add_delay(self, delay_time):
        self.delayed_time = delay_time
        self.estimated_exit_time += delay_time
        self.incident_count += 1
class Event:
    def __init__(self, timestamp, event_type, vehicle= None, link = None, link_length = None):
        self.timestamp = timestamp
        self.event_type = event_type
        self.vehicle = vehicle
        self.link = link
        self.link_length = link_length
    def __lt__(self, other):
        return self.timestamp < other.timestamp

In [5]:
assignmentGraph = nx.read_gml('networkAssignment.gml')

# Define the arrival rates per hour (vehicles per hour)
arrival_rates = {
    0: 314.2, 1: 162.4, 2: 138.6, 3: 148.8, 4: 273.2, 5: 1118.8, 6: 2773.8, 7: 4036.2,
    8: 4237.4, 9: 3277.0, 10: 2843.0, 11: 2876.4, 12: 3143.0, 13: 3277.8, 14: 3546.2, 15: 4335.0,
    16: 4945.4, 17: 4525.8, 18: 2847.8, 19: 1828.0, 20: 1378.4, 21: 1271.2, 22: 1171.2, 23: 767.6
}

# Define vehicle types and their properties
vehicle_types = {
    'car': {'probability': 0.9, 'max_speed': 100},  # 90% cars, max speed 100 km/h
    'truck': {'probability': 0.1, 'max_speed': 80}  # 10% trucks, max speed 80 km/h
}

# Define City A and City B
city_A = '43108886'
city_B = '44996729'

# Function to calculate travel time for a link
def calculate_travel_time(link_length, max_speed):
    mean_travel_time = link_length / max_speed  # in hours
    std_dev = mean_travel_time / 20  # standard deviation
    travel_time = norm.rvs(loc=mean_travel_time, scale=std_dev)  # normally distributed
    return travel_time * 60  # convert to minutes

In [6]:
def simulate_traffic_fes(G, arrival_rates, vehicle_types, city_A, city_B,incident_rates, incident_durations, fract_nav, num_runs=10):
    fes = FES()
    nodes = list(G.nodes)
    curren_time = 0
    veh_on_link = defaultdict(list)
    for hour in range(24):
        rate = arrival_rates[hour]
        num_vehicles = np.random.poisson(rate)
        
        current_time = hour*60
        inc_rate = incident_rates[hour]
        num_incidents = np.random.poisson(inc_rate)
        for _ in range(num_incidents):
            u,w = random.choice(list(G.edges))
            start_incident = np.random.uniform(0,60)
            link_length = G[u][w]['length']
            incident_time = current_time+ start_incident
            fes.add(Event(incident_time,'start incident', link =(u,w) ))
            incident_duration = np.random.normal(0, incident_durations[hour]) #depends on data
            end_incident = incident_time + incident_duration

            fes.add(Event(end_incident, ' end incident', link = (u,w),link_length= link_length))

            


             
        for _ in range(num_vehicles):
            # Randomly select origin and destination
            origin, destination = np.random.choice(nodes, size=2, replace=False)

            # Randomly select vehicle type
            vehicle_type = np.random.choice(list(vehicle_types.keys()), p=[v['probability'] for v in vehicle_types.values()])
            reroutes = np.random.rand() < fract_nav
            path = nx.shortest_path(G,origin, destination,weight = 'length')
            inter_arrival_time = np.random.exponential(scale=60 / rate)
            arrival_time = current_time + inter_arrival_time
            v = Vehicles(origin, destination, vehicle_type, path, reroutes)
            fes.add(Event(arrival_time,'arrival',v))
            
    while not fes.isEmpty():
                
                event = fes.next()
                time = event.timestamp
                
                if event.event_type == 'arrival':
                    vehicle = event.vehicle
                    vehicle.link_idx = 0
                    u,w = vehicle.current_link
                    if G[u][w]['incident_active'] == True:
                         G[u][w]['queue'].append(vehicle)
                    vehicle.update_position(time,fes, veh_on_link,G)

                elif event.event_type == 'move_next_link':
                     vehicle = event.vehicle
                     u,w = vehicle.current_link

                     if G[u][w]['incident_active']== True and vehicle.reroutes:
                          u = vehicle.current_link[0] 
                          w = vehicle.destination
                          path = nx.shortest_path(G, source=u, target=v, weight='length')
                          vehicle.path = path

                     if G[u][w]['incident_active']== True and vehicle in G[u][w]['queue']:
                        continue
                     elif G[u][w]['incident_active']== True:
                          G[u][w]['queue'].append(vehicle)
                     if vehicle in veh_on_link[(u, w)]:
                        veh_on_link[(u, w)].remove(vehicle)
                     
                     vehicle.link_idx +=1
                     vehicle.update_position(time,fes, veh_on_link,G)
                
                     
                elif  event.event_type == 'start_incident':
                    u,w= event.link
                    G[u][w] ['incident_active']= True
                    original_length = G[u][w]['length']
                    G[u][w]['length'] = float('inf')
                    G[u][w]['queue'] = deque()
                    link_length = event.link_length 
                    affected_vehicle = []
                    

                    for veh in veh_on_link[(u,w)]:
                         if veh.link_entry_time<= time <= (veh.link_entry_time + 0.75*link_length/veh.v_max):  
                            affected_vehicle.append(veh)
                            
                    G[u][w]['queue'] = deque(affected_vehicle)
                    for veh in affected_vehicle:
                         veh.add_delay()
                     
                
                elif event.event_type == 'end incident':
                    u,w = event.link
                    G[u][w] ['incident_active']= False
                    G[u][w]['length'] = original_length
                    queue = G[u][w].get('queue', deque())
                    discharge_rate = G[u][w].get('lanes')
                    discharge_time = time

                    while queue:
                        vehicle = queue.popleft()
                        discharge_time += 1/discharge_rate
                
                        if vehicle in veh_on_link[(u, w)]:
                            veh_on_link[(u, w)].remove(vehicle)
                        
                        vehicle.link_idx +=1
                        vehicle.update_position(discharge_time,fes, veh_on_link,G)